In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [15]:
data=pd.read_csv("customercampaign.csv")

In [16]:
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,4/9/2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,8/3/2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21/8/2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10/2/2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19/1/2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [18]:
data["Income"] = data["Income"].fillna(data["Income"].mean())


data["Income"] = data["Income"].astype(int)


In [19]:
data['income_group'] = pd.qcut(data['Income'], q=4, labels=['Low', 'Medium', 'High', 'High-plus'])

data['income_group_num'] = data['income_group'].cat.codes + 1


print(data[['Income', 'income_group', 'income_group_num']].head(10))

   Income income_group  income_group_num
0   58138         High                 3
1   46344       Medium                 2
2   71613    High-plus                 4
3   26646          Low                 1
4   58293         High                 3
5   62513         High                 3
6   55635         High                 3
7   33454          Low                 1
8   30351          Low                 1
9    5648          Low                 1


In [20]:
education_map = {
    'Basic': 1,
    '2n Cycle': 2,
    'Graduation': 3,
    'Master': 4,
    'PhD': 5
}

data['EducationLevel'] = data['Education'].map(education_map)

In [21]:
data = data[~data['Marital_Status'].isin(['Absurd', 'YOLO'])].copy()
data['Marital_Status'] = data['Marital_Status'].replace({'Alone': 'Single'})

In [22]:
print(data['Marital_Status'].value_counts())

marital_mapping = {
    'Single': 1,
    'Divorced': 1,
    'Widow': 1,
    'Married': 2,
    'Together': 2,
}

data['Marital_Status_Encoded'] = data['Marital_Status'].map(marital_mapping)


Marital_Status
Married     864
Together    580
Single      483
Divorced    232
Widow        77
Name: count, dtype: int64


In [23]:
data['has_kids'] = ((data['Kidhome'] > 0) | (data['Teenhome'] > 0)).astype(int)

In [24]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'], errors='coerce').dt.normalize()


data['month_year'] = data['Dt_Customer'].dt.to_period('M').astype(str)
data['year']       = data['Dt_Customer'].dt.year
data['month']      = data['Dt_Customer'].dt.month
data['half_year']  = data['month'].apply(lambda x: 'First Half' if x <= 6 else 'Second Half')
data['year_half']  = data['year'].astype(str) + ' - ' + data['half_year']
data = data.drop(columns=['year', 'month', 'half_year'])


data = data[data['Year_Birth'].between(1900, 2005)]
data['Age'] = data['Dt_Customer'].dt.year - data['Year_Birth']
print(data[['Dt_Customer', 'month_year', 'year_half']].head())

data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna(subset=['Age'])

data = data[(data['Age'] >= 18) & (data['Age'] <= 90)]

  Dt_Customer month_year             year_half
0  2012-04-09    2012-04   2012.0 - First Half
1  2014-08-03    2014-08  2014.0 - Second Half
2         NaT        NaT     nan - Second Half
3  2014-10-02    2014-10  2014.0 - Second Half
4         NaT        NaT     nan - Second Half


In [25]:
data.drop(columns=['Education', 'Marital_Status', 'Kidhome', 'Teenhome', 'Year_Birth', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue','income_group'], inplace=True)

In [26]:
data

,ID,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp2,Complain,Response,income_group_num,EducationLevel,Marital_Status_Encoded,has_kids,month_year,year_half,Age
0,5524,58138,58,635,88,546,172,88,88,3,...,0,0,1,3,3,1,0,2012-04,2012.0 - First Half,55.0
1,2174,46344,38,11,1,6,2,1,6,2,...,0,0,0,2,3,1,1,2014-08,2014.0 - Second Half,60.0
3,6182,26646,26,11,4,20,10,3,5,2,...,0,0,0,1,3,2,1,2014-10,2014.0 - Second Half,30.0
5,7446,62513,16,520,42,98,0,42,14,2,...,0,0,0,3,4,2,1,2013-09,2013.0 - Second Half,46.0
7,6177,33454,32,76,10,56,3,1,23,2,...,0,0,0,1,5,2,1,2013-08,2013.0 - Second Half,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,4201,57967,39,229,7,137,4,0,91,5,...,0,0,0,3,3,1,1,2013-03,2013.0 - First Half,51.0
2228,8720,52247,53,32,2,1607,12,4,22,0,...,0,0,0,3,2,2,0,2012-12,2012.0 - Second Half,34.0
2233,9432,666666,23,9,14,18,8,1,12,4,...,0,0,0,4,3,2,1,2013-02,2013.0 - First Half,36.0
2234,8372,34421,81,3,3,7,6,2,9,1,...,0,0,0,1,3,2,1,2013-01,2013.0 - First Half,39.0
